# Decision Trees


<!-- ## Naive ideas -->

Given a dataset with labels, the decision tree algorithm firstly trys to split the whole dataset into two different groups, based on some speicific features. Choose which feature to use and set the threshold for the split are done.



## Gini impurity

To split a dataset, we need a metric to tell whether the split is good or not. The two most popular metrics that are used are Gini impurity and Entropy. The two metrics don't have essential differences, that the results obtained by applying each metric are very similar to each other. Therefore we will only focus on Gini impurity since it is slightly easier to compute and slightly easier to explain.

### Motivation and Definition
Assume that we have a dataset of totally $n$ objects, and these objects are divided into $k$ classes. The $i$-th class has $n_i$ objects. Then if we randomly pick an object, the probability to get an object belonging to the $i$-th class is

$$
p_i=\frac{n_i}{n}
$$

If we then guess the class of the object purely based on the distribution of each class, the probability that our guess is incorrect is 

$$
1-p_i = 1-\frac{n_i}{n}.
$$

Therefore, if we randomly pick an object that belongs to the $i$-th class and randomly guess its class purely based on the distribution but our guess is wrong, the probability that such a thing happens is 

$$
p_i(1-p_i).
$$

Consider all classes. The probability at which any object of the dataset will be mislabelled when it is randomly labeled is the sum of the probability described above:

$$
\sum_{i=1}^kp_i(1-p_i)=\sum_{i=1}^kp_i-\sum_{i=1}^kp_i^2=1-\sum_{i=1}^kp_i^2.
$$

This is the definition formula for the *Gini impurity*. 



::: {#def-gini}
The **Gini impurity** is calculated using the following formula

$$
Gini = \sum_{i=1}^kp_i(1-p_i)=\sum_{i=1}^kp_i-\sum_{i=1}^kp_i^2=1-\sum_{i=1}^kp_i^2,
$$
where $p_i$ is the probability of class $i$.
:::

The way to understand Gini impurity is to consider some extreme examples. 


::: {#exm-}

Assume that we only have one class. Therefore $k=1$, and $p_1=1$. Then the Gini impurity is

$$
Gini = 1-1^2=0.
$$
This is the minimum possible Gini impurity. It means that the dataset is **pure**: all the objects contained are of one unique class. In this case, we won't make any mistakes if we randomly guess the label.

:::




::: {#exm-}
Assume that we have two classes. Therefore $k=2$. Consider the distribution $p_1$ and $p_2$. We know that $p_1+p_2=1$. Therefore $p_2=1-p_1$. Then the Gini impurity is

$$
Gini(p_1) = 1-p_1^2-p_2^2=1-p_1^2-(1-p_1)^2=2p_1-2p_1^2.
$$
When $0\leq p_1\leq 1$, this function $Gini(p_1)$ is between $0$ and $0.5$. 
- It gets $0$ when $p_1=0$ or $1$. In these two cases, the dataset is still a one-class set since the size of one class is $0$. 
- It gets $0.5$ when $p_1=0.5$. This means that the Gini impurity is maximized when the size of different classes are balanced.
:::

### Algorithm


::: {.callout-note}
# Algorithm: Gini impurity

**Inputs** A dataset $S=\{data=[features, label]\}$ with labels. 

**Outputs** The Gini impurity of the dataset.

1. Get the size $n$ of the dataset.
2. Go through the label list, and find all unique labels: $uniqueLabelList$.
3. Go through each label $l$ in $uniqueLabelList$ and count how many elements belonging to the label, and record them as $n_l$.
4. Use the formula to compute the Gini impurity:

   $$
    Gini = 1-\sum_{l\in uniqueLabelList}\left(\frac{n_l}{n}\right)^2.
   $$
:::


The sample codes are listed below:

In [ ]:
import pandas as pd
def gini(S):
    N = len(S)
    y = S[:, -1].reshape(N)
    gini = 1 - ((pd.Series(y).value_counts()/N)**2).sum()
    return gini

## CART Algorithms 

### Ideas
Consider a labeled dataset $S$ with totally $m$ elements. We use a feature $k$ and a threshold $t_k$ to split it into two subsets: $S_l$ with $m_l$ elements and $S_r$ with $m_r$ elements. Then the cost function of this split is

$$
J(k, t_k)=\frac{m_l}mGini(S_l)+\frac{m_r}{m}Gini(S_r).
$$
It is not hard to see that the more pure the two subsets are the lower the cost function is. Therefore our goal is find a split that can minimize the cost function.

:::{.callout-note}

# Algorithm: Split the Dataset

**Inputs** Given a labeled dataset $S=\{[features, label]\}$.

**Outputs** A best split $(k, t_k)$.

1. For each feature $k$:
    1. For each value $t$ of the feature:
        1. Split the dataset $S$ into two subsets, one with $k\leq t$ and one with $k>t$.
        2. Compute the cost function $J(k,t)$. 
        3. Compare it with the current smallest cost. If this split has smaller cost, replace the current smallest cost and pair with $(k, t)$.
2. Return the pair $(k,t_k)$ that has the smallest cost function.
:::


We then use this split algorithm recursively to get the decision tree.

:::{.callout-note}
# Classification and Regression Tree, CART

**Inputs** Given a labeled dataset $S=\{[features, label]\}$ and a maximal depth `max_depth`.

**Outputs** A decision tree.

1. Starting from the original dataset $S$. Set the working dataset $G=S$.
2. Consider a dataset $G$. If $Gini(G)\neq0$, split $G$ into $G_l$ and $G_r$ to minimize the cost function. Record the split pair $(k, t_k)$.
3. Now set the working dataset $G=G_l$ and $G=G_r$ respectively, and apply the above two steps to each of them.
4. Repeat the above steps, until `max_depth` is reached.
:::

Here are the sample codes.

In [ ]:
def split(G):
    m = G.shape[0]
    gmini = gini(G)
    pair = None
    if gini(G) != 0:
        numOffeatures = G.shape[1] - 1
        for k in range(numOffeatures):
            for t in range(m):
                Gl = G[G[:, k] <= G[t, k]]
                Gr = G[G[:, k] > G[t, k]]
                gl = gini(Gl)
                gr = gini(Gr)
                ml = Gl.shape[0]
                mr = Gr.shape[0]
                g = gl*ml/m + gr*mr/m
                if g < gmini:
                    gmini = g
                    pair = (k, G[t, k])
                    Glm = Gl
                    Grm = Gr
        res = {'split': True,
               'pair': pair,
               'sets': (Glm, Grm)}
    else:
        res = {'split': False,
               'pair': pair,
               'sets': G}
    return res

For the purpose of counting labels, we also write a code to do so.

In [ ]:
import pandas as pd
def countlabels(S):
    y = S[:, -1].reshape(S.shape[0])
    labelCount = dict(pd.Series(y).value_counts())
    return labelCount

## Decision Tree Project 1: the `iris` dataset

We are going to use the Decision Tree model to study the `iris` dataset. This dataset has already studied previously using k-NN. Again we will only use the first two features for visualization purpose.

### Initial setup

Since the dataset will be splitted, we will put `X` and `y` together as a single variable `S`. In this case when we split the dataset by selecting rows, the features and the labels are still paired correctly. 

We also print the labels and the feature names for our convenience.


In [ ]:
from sklearn.datasets import load_iris
import numpy as np
from assests.codes.dt import gini, split, countlabels

iris = load_iris()
X = iris.data[:, 2:]
y = iris.target
y = y.reshape((y.shape[0],1))
S = np.concatenate([X,y], axis=1)

print(iris.target_names)
print(iris.feature_names)

### Apply CART manually

We apply `split` to the dataset `S`. 


In [ ]:
r = split(S)
if r['split'] is True:
    Gl, Gr = r['sets']
    print(r['pair'])
    print('The left subset\'s Gini impurity is {g:.2f},'.format(g=gini(Gl)),
          ' and its label counts is {d:}'.format(d=countlabels(Gl)))
    print('The right subset\'s Gini impurity is {g:.2f},'.format(g=gini(Gr)),
          ' and its label counts is {d}'.format(d=countlabels(Gr)))

The results shows that `S` is splitted into two subsets based on the `0`-th feature and the split value is `1.9`. 

The left subset is already pure since its Gini impurity is `0`. All elements in the left subset is label `0` (which is `setosa`). The right one is mixed since its Gini impurity is `0.5`. Therefore we need to apply `split` again to the right subset.


In [ ]:
r = split(Gr)
if r['split'] is True:
    Grl, Grr = r['sets']
    print(r['pair'])
    print('The left subset\'s Gini impurity is {g:.2f},'.format(g=gini(Grl)),
          ' and its label counts is {d:}'.format(d=countlabels(Grl)))
    print('The right subset\'s Gini impurity is {g:.2f},'.format(g=gini(Grr)),
          ' and its label counts is {d}'.format(d=countlabels(Grr)))

This time the subset is splitted into two more subsets based on the `1`-st feature and the split value is `1.7`. The total Gini impurity is minimized using this split. 

The decision we created so far can be described as follows:

1. Check the first feature `sepal length (cm)` to see whether it is smaller or equal to `1.9`.
   1. If it is, classify it as lable `0` which is `setosa`.
   2. If not, continue to the next stage.
2. Check the second feature `sepal width (cm)` to see whether it is smaller or equal to `1.7`. 
   1. If it is, classify it as label `1` which is `versicolor`.
   2. If not, classify it as label `2` which is `virginica`.

### Use package `sklearn`

Now we would like to use the decision tree package provided by `sklearn`. The process is straightforward. The parameter `random_state=40` will be discussed {ref}`later<note-random_state>`, and it is not necessary in most cases.


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=2, random_state=40)
clf.fit(X, y)

`sklearn` provide a way to automatically generate the tree view of the decision tree. The code is as follows. 


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(2, 2), dpi=200)
tree.plot_tree(clf, filled=True, impurity=True)

Similar to k-NN, we may use `sklearn.inspection.DecisionBoundaryDisplay` to visualize the decision boundary of this decision tree.


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
DecisionBoundaryDisplay.from_estimator(
    clf,
    X,
    cmap='coolwarm',
    response_method="predict",
    xlabel=iris.feature_names[0],
    ylabel=iris.feature_names[1],
)

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, s=15)

### Analyze the differences between the two methods
The tree generated by `sklearn` and the tree we got manually is a little bit different. Let us explore the differences here. 

To make it easier to split the set, we could convert the `numpy.ndarray` to `pandas.DataFrame`.


In [ ]:
import pandas as pd

df = pd.DataFrame(X)
df.head()

Now based on our tree, we would like to get all data points that the first feature (which is marked as `0`) is smaller or equal to `1.9`. We save it as `df1`. Similarly based on the tree gotten from `sklearn`, we would like to get all data points taht the second feature (which is marked as `1`) is smaller or equal to `0.8` and save it to `df2`. 


In [ ]:
df1 = df[df[0]<=1.9]
df2 = df[df[1]<=0.8]

Then we would like to compare these two dataframes. What we want is to see whether they are the same regardless the order. One way to do this is to sort the two dataframes and then compare them directly.

To sort the dataframe we use the method `DataFrame.sort_values`. The details can be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html). Note that after `sort_values` we apply `reset_index` to reset the index just in case the index is massed by the sort operation.

Then we use `DataFrame.equals` to check whether they are the same.


In [ ]:
df1sorted = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
df2sorted = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
print(df1sorted.equals(df2sorted))

So these two sets are really the same. The reason this happens can be seen from the following two graphs.


:::{layout-ncol=2}


![From our code](assests/img/20220809120531.png)

![From `sklearn`](assests/img/20220809122643.png)

:::



So you can see that either way can give us the same classification. This means that given one dataset the construction of the decision tree might be random at some points.


::: {.callout-note}
# note-random_state
Since the split is random, when using `sklearn.DecisionTreeClassifier` to construct decision trees, sometimes we might get the same tree as what we get from our naive codes. 

To illustrate this phenomenaon I need to set the random state in case it will generate the same tree as ours when I need it to generate a different tree. The parameter `random_state=40` mentioned before is for this purpose.
:::



Another difference is the split value of the second branch. In our case it is `1.7` and in `sklearn` case it is `1.75`. So after we get the right subset from the first split (which is called `dfr`), we would split it into two sets based on whether the second feature is above or below `1.7`.


In [ ]:
dfr = df[df[0]>1.9]
df2a = dfr[dfr[1]>1.7]
df2b = dfr[dfr[1]<=1.7]
print(df2b[1].max())
print(df2a[1].min())

Now you can see where the split number comes from. In our code, when we found a split, we will directly use that number as the cut. In this case it is `1.7`. 

In `sklearn`, when it finds a split, the algorithm will go for the middle of the gap as the cut. In this case it is `(1.7+1.8)/2=1.75`. 

## Decision Tree Project 2: `make_moons` dataset

`sklearn` includes various random sample generators that can be used to build artificial datasets of controlled size and complexity. We are going to use `make_moons` in this section. More details can be found [here](https://scikit-learn.org/stable/datasets/sample_generators.html).

`make_moons` generate 2d binary classification datasets that are challenging to certain algorithms (e.g. centroid-based clustering or linear classification), including optional Gaussian noise. `make_moons` produces two interleaving half circles. It is useful for visualization. 

Let us explorer the dataset first.


In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
plt.scatter(x=X[:, 0], y=X[:, 1], c=y)

Now we are applying `sklearn.DecisionTreeClassifier` to construct the decision tree. The steps are as follows.

1. Split the dataset into training data and test data. 
2. Construct the pipeline. Since we won't apply any transformers there for this problem, we may just use the classifier `sklearn.DecisionTreeClassifier` directly without really construct the pipeline object.
3. Consider the hyperparameter space for grid search. For this problme we choose `min_samples_split` and `max_leaf_nodes` as the hyperparameters we need. We will let `min_samples_split` run through 2 to 5, and `max_leaf_nodes` run through 2 to 50. We will use `grid_search_cv` to find the best hyperparameter for our model. For cross-validation, the number of split is set to be `3` which means that we will run trainning 3 times for each pair of hyperparameters.
4. Run `grid_search_cv`. Find the best hyperparameters and the best estimator. Test it on the test set to get the accuracy score.


In [ ]:
# Step 1
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 3
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np

params = {'min_samples_split': list(range(2, 5)),
          'max_leaf_nodes': list(range(2, 50))}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), 
                              params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

In [ ]:
# Step 4
from sklearn.metrics import accuracy_score

clf = grid_search_cv.best_estimator_
print(grid_search_cv.best_params_)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

Now you can see that for this `make_moons` dataset, the best decision tree should have at most `17` leaf nodes and the minimum number of samples required to be at a leaft node is `2`. The fitted decision tree can get 86.95% accuracy on the test set. 

Now we can plot the decision tree and the decision surface.


In [ ]:
from sklearn import tree
plt.figure(figsize=(15, 15), dpi=300)
tree.plot_tree(clf, filled=True)

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

DecisionBoundaryDisplay.from_estimator(
    clf,
    X,
    cmap=plt.cm.RdYlBu,
    response_method="predict"
)
plt.scatter(
    X[:, 0],
    X[:, 1],
    c=y,
    cmap='gray',
    edgecolor="black",
    s=15,
    alpha=.15)

Since it is not very clear what the boundary looks like, I will draw the decision surface individually below.

In [ ]:
DecisionBoundaryDisplay.from_estimator(
    clf,
    X,
    cmap=plt.cm.RdYlBu,
    response_method="predict"
)

## Exercises and Projects


::: {#exr-}
The dataset and its scattering plot is given below.

1. Please calculate the Gini impurity of the whole set by hand.
2. Please apply CART to create the decision tree by hand. 
3. Please use the tree you created to classify the following points:
    - $(0.4, 1.0)$
    - $(0.6, 1.0)$
    - $(0.6, 0)$

The following code is for ploting. You may also get the precise data points by reading the code. You don't need to write codes to solve the problem.

In [ ]:
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
data = {'x0': [0.22, 0.37, 0.42, 0.45, 0.18, 0.20, 0.21, 0.23, 0.35, 0.58,
               0.60, 0.61, 0.62, 0.65, 0.70, 0.75, 0.82, 0.88, 0.90, 0.92],
        'x1': [0.83, 0.78, 0.65, 0.37, 0.57, 0.45, 0.67, 0.22, 0.43, 0.33,
               0.75, 0.50, 0.21, 0.31, 0.64, 0.70, 0.80, 0.82, 0.61, 0.81],
        'y': ['r', 'r', 'b', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 
              'b', 'b', 'r', 'r', 'b', 'r', 'r', 'r', 'r', 'r']}
df = pd.DataFrame(data)

plt.figure(figsize=(7, 7))
plt.scatter(df['x0'], df['x1'], c=df['y'])
_ = plt.xlim(0, 1)
_ = plt.ylim(0, 1)

:::






::: {#exr-}
CHOOSE ONE: Please apply the Decision Tree to one of the following datasets. 

- dating dataset (in Chpater 2). 
- the `titanic` dataset.

Please answer the following questions.

1. Please use grid search to find the good `max_leaf_nodes` and `max_depth`.
2. Please record the accuracy (or cross-validation score) of your model and compare it with the models you learned before (kNN). 
3. Please find the two most important features and explane your reason.
4. (Optional) Use the two most important features to draw the Decision Boundary if possible.
:::
